In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pd.options.display.max_columns = 1000

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-mar-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-mar-2021/test.csv')

In [ ]:
df_train = train.copy()
df_train = pd.get_dummies(df_train, columns=['cat0','cat11', 'cat12', 'cat13', 'cat14',
                                  'cat15', 'cat16', 'cat17', 'cat18'], drop_first=True)

In [ ]:
train_target = df_train['target']
df_train = df_train.drop('target', axis=1)
df_train = df_train.drop('id', axis=1)

In [ ]:
from sklearn.preprocessing import LabelEncoder
cat = ['cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9']
for i in cat:
    lbl = LabelEncoder()
    df_train[i] = lbl.fit_transform(list(df_train[i].values))

In [ ]:
cat = ['cat10']
for c in cat:
    freq = df_train[c].value_counts()
    df_train[c] = df_train[c].map(freq)

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
train_x,varid_x,train_y,varid_y = train_test_split(df_train,
                                                   train_target,
                                                   test_size=0.33,
                                                   random_state=0)

In [ ]:
varid_x.shape

In [ ]:
# tmp = df_train.append(test, ignore_index=False, sort=False)
# categorical_feature = np.where(temp.dtypes != )
lgb_train = lgb.Dataset(train_x,train_y)
lgb_eval =lgb.Dataset(varid_x,varid_y)

lgbm_params = {
#    'cat_feature': categorical_feature,
    'metric': 'auc',
    'n_estimators' : 20000,
    'reg_alpha': 0.000721024661208569,
    'reg_lambda': 47.79748127808107,
    'colsample_bytree': 0.24493010466517195,
    'subsample': 0.12246675404710294,
    'learning_rate': 0.013933182980403087,
    'max_depth': 21,
    'num_leaves': 90,
    'min_child_samples': 144,
    'cat_smooth': 63
}


evals_result = {}
gbm = lgb.train(
    params = lgbm_params,
    train_set = lgb_train,
    valid_sets = [lgb_train,lgb_eval],
    early_stopping_rounds = 20,
    evals_result = evals_result,
    verbose_eval = 10
)

In [ ]:
oof = (gbm.predict(varid_x) > 0.5).astype(int)
print('score',round(accuracy_score(varid_y,oof)*100,2))

In [ ]:
import matplotlib.pyplot as plt 
plt.style.use('seaborn-whitegrid')
%matplotlib inline
plt.plot(evals_result['training']['auc'],label = 'train_loss')
plt.plot(evals_result['valid_1']['auc'],label = 'valid_loss')
plt.legend()
plt.show()

In [ ]:
df_test = test.copy()
df_test = df_test.drop('id', axis=1)
df_test = pd.get_dummies(df_test, columns=['cat0','cat11', 'cat12', 'cat13', 'cat14',
                                  'cat15', 'cat16', 'cat17', 'cat18'], drop_first=True)

cat = ['cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9']
for i in cat:
    lbl = LabelEncoder()
    df_test[i] = lbl.fit_transform(list(df_test[i].values))
    
cat = ['cat10']
for c in cat:
    freq = df_test[c].value_counts()
    df_test[c] = df_test[c].map(freq)

In [ ]:
df_test.tail()

In [ ]:
sample_submission = pd.read_csv('../input/tabular-playground-series-mar-2021/sample_submission.csv')
test_pred = (gbm.predict(df_test)>0.5).astype(int)
sample_submission['target'] = test_pred

In [ ]:
sample_submission.head()

In [ ]:
sample_submission.to_csv('sample_2.csv', index=False)